In [1]:
import dotenv
import json
import math
import os
import sys
import typing as T

import plotly.express as px

current_dir = %pwd
root_dir = os.path.dirname(current_dir)
src_dir = os.path.join(root_dir, 'src')
sys.path.append(src_dir)

dotenv.load_dotenv()

True

In [2]:
from demographics.google_places import GooglePlacesAPI
from demographics.util import get_viewport, get_grid_coordinates

API_KEY = os.getenv("GOOGLE_MAPS_PLACES_API_KEY")

assert API_KEY, "API_KEY not found in environment variables"

METERS_PER_MILE = 1609.34

In [3]:
google_places = GooglePlacesAPI(API_KEY, verbose=True)

In [4]:
def save_json(filename, prompt: str, fields: T.List[str], data: T.Dict[str, T.Any]):
    if "places" not in data:
        print("No places found")
        print(f"Data: {json.dumps(data, indent=2)}")
        return

    print(f"Number of results: {len(results['places'])}")

    print(f"Saving to {filename}")
    with open(filename, "w") as f:
        data = {
            "prompt": prompt,
            "fields": fields,
            "results": results
        }
        json.dump(data, f, indent=2)
import math


In [5]:
ADVANCED_FIELDS = [
    "places.formattedAddress",
    "places.displayName",
    "places.nationalPhoneNumber",
    "places.location",
    "places.rating",
    "places.googleMapsUri",
    "places.websiteUri",
    "places.regularOpeningHours",
    "places.businessStatus",
    "places.priceLevel",
    "places.userRatingCount",
    "places.takeout",
    "places.delivery",
    "places.dineIn",
    "places.servesBreakfast",
    "places.primaryTypeDisplayName",
    "places.primaryType",
    "places.editorialSummary",
    "places.outdoorSeating",
    "places.servesCoffee",
    "places.paymentOptions",
    "places.accessibilityOptions",
]

TYPES = [
    "bakery",
    "sandwich_shop",
    "coffee_shop",
    "cafe",
    "fast_food_restaurant",
    "store",
    "restaurant",
    "food",
    "point_of_interest",
    "establishment",
]
ADVANCED_PROMPT = f"All {'s, '.join([t for t in TYPES])}"

print(ADVANCED_PROMPT)


All bakerys, sandwich_shops, coffee_shops, cafes, fast_food_restaurants, stores, restaurants, foods, point_of_interests, establishment


In [6]:
def calculate_cost_from_results(results: T.Dict[str, T.Any], cost_per_square: float, radius_miles: float) -> T.Tuple[int, float]:
    meters_per_mile = 1609.34  # One mile in meters
    search_block_area = 500 * 500  # Area of one square in square meters

    # Convert radius from miles to meters
    radius_meters = radius_miles * meters_per_mile
    area_width = radius_meters * 2

    total_area_meters = area_width * area_width

    # Calculate how many 50 meter squares fit into the area
    number_of_squares = total_area_meters / search_block_area

    total_cost = number_of_squares * cost_per_square

    return number_of_squares, total_cost

# Conversion factors and constants
meters_per_mile = 1609.34  # One mile in meters
radius_miles = 20  # Radius in miles
search_block_area = 500 * 500  # Area of one square in square meters

# Convert radius from miles to meters
radius_meters = radius_miles * meters_per_mile
area_width = radius_meters * 2

total_area_meters = area_width * area_width

# Calculate how many 50 meter squares fit into the area
number_of_squares = total_area_meters / search_block_area

costs_per_square = 0.04  # Cost per square in USD

total_cost = number_of_squares * costs_per_square

print(f"Total searches: {number_of_squares:.0f}")
print(f"Total cost: ${total_cost:.2f}")
print(f"Searched area: {search_block_area:.2f} m^2")
print(f"Total area: {total_area_meters:.2f} m^2")


Total searches: 16576
Total cost: $663.03
Searched area: 250000.00 m^2
Total area: 4143960376.96 m^2


In [7]:
locations = {
    "Portland": {
        "latitude": 45.54,
        "longitude": -122.98,
    },
    "New York": {
        "latitude": 40.71,
        "longitude": -74.01,
    },
}


In [8]:
location = locations["Portland"]
radius_miles = 20.0
radius_meters = radius_miles * METERS_PER_MILE
grid_sqft = 500.0

grid = get_grid_coordinates(location["latitude"], location["longitude"], radius_meters, grid_sqft)

print(f"Grid size: {len(grid)}")
print(f"First 5 grid points: {grid[:5]}")

lat_steps: 128, lon_steps: 128
Grid size: 16129
First 5 grid points: [(45.25264092712729, -123.38960677930046), (45.25264092712729, -123.38319400815088), (45.25264092712729, -123.37678123700128), (45.25264092712729, -123.3703684658517), (45.25264092712729, -123.3639556947021)]


In [ ]:
prompt = "All restaurants"
fields = ADVANCED_FIELDS
radius_miles = 0.5
# radius_meters = radius_miles * METERS_PER_MILE
radius_meters = 150.0
location = "New York"

latitude = locations[location]["latitude"]
longitude = locations[location]["longitude"]

rect_viewpoint = get_viewport(latitude, longitude, radius_meters)

data = {
    "locationRestriction": {
        "rectangle": rect_viewpoint
    }
}
results = google_places.text_search(prompt, fields, data)
if "places" not in results:
    print("No results found")
else:
    filename = f"{latitude}_{longitude}_{radius_miles}_location_rect_viewpoint_search.json"
    save_json(filename, prompt, fields, results)


In [ ]:
radius_miles = 0.5
latitude = 45.54
longitude = -122.98
prompt = ADVANCED_PROMPT
fields = ADVANCED_FIELDS

results = google_places.search_location_radius(latitude, longitude, radius_miles, prompt, ADVANCED_FIELDS)
filename = f"{latitude}_{longitude}_{radius_miles}_location_radius_search.json"
save_json(filename, prompt, fields, results)


In [ ]:
radius_miles = 1
location = "San Francisco"
prompt = f"All bakeries within {radius_miles} miles of the center of {location}"
fields = ["*"]

results = google_places.text_search(prompt, fields)
filename = f"{location}_{radius_miles}_text_search.json"
save_json(filename, prompt, fields, results)

In [ ]:
radius_miles = 1
location = "San Francisco"
prompt = f"All bakeries within {radius_miles} miles of the center of {location}"
fields = ADVANCED_FIELDS

results = google_places.text_search(prompt, fields)
filename = f"{location}_{radius_miles}_text_search.json"
save_json(filename, prompt, fields, results)

In [ ]:
prompt = "All restaurants"
fields = ADVANCED_FIELDS
data = {
    "locationBias": {
        "circle": {
            "center": {
                "latitude": 45.54,
                "longitude": -122.98
            },
            "radius": 10.0
        }
    }
}
results = google_places.text_search(prompt, fields, data)
filename = f"{latitude}_{longitude}_{radius_miles}_location_radius_search.json"
save_json(filename, prompt, fields, results)